In [ ]:
import os
import sys
import requests
from datetime import datetime 
from multiprocessing import Pool
import numpy as  np
import pandas as pd
from pathlib import Path
import pickle
from tqdm import tqdm
from time import time
from typing import Dict, List, Optional
sys.path.append('../')
sys.path.append('../anmodel')
import matplotlib.pyplot as plt
import anmodel
import matplotlib
import scipy
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'

In [ ]:
S=pd.read_csv("A.csv")
anan1=anmodel.models.NAN10model()
num=242

A=[]
A.append(S["g_kvhh"][num])
A.append(S["g_unav"][num])
A.append(S["g_kna"][num])
A.append(S["g_leak"][num])
A.append(S["t_na"][num])
A.append(S["x_na"][num])
A.append(S["y_na"][num])
A.append(S["g_cav"][num])


BBB=['g_kvhh', 'g_unav', 'g_kna', 'g_leak',  't_na', 'x_na', 'y_na', 'g_cav']
dic={key: val for key , val in zip(BBB,A)}
new_params=dic
new_params

In [ ]:
S=pd.read_csv("A.csv")
anan1=anmodel.models.NAN10dmodel()
num=242

A=[]
A.append(S["g_kvhh"][num])
A.append(S["g_unav"][num])
A.append(S["g_kna"][num])
A.append(S["g_leak"][num]*0.6095)
A.append(S["g_leak"][num]*0.3905)
A.append(S["t_na"][num])
A.append(S["x_na"][num])
A.append(S["y_na"][num])
A.append(S["g_cav"][num])


BBB=['g_kvhh', 'g_unav', 'g_kna', 'g_lek','g_lena',  't_na', 'x_na', 'y_na', 'g_cav']
dic={key: val for key , val in zip(BBB,A)}
new_params=dic
anan1.set_params(new_params)
s,info=anan1.run_odeint()
plt.plot(s[400000:,0])

In [ ]:
#Figure2

In [ ]:
t1=[]
for i in range(int(0.1*1000*10)):
    t1.append(0.001*0.1*i)
C1=[]
D1=[]
for i in range(len(t1)):
    C1.append(s[486000:,0][i+1800])
    D1.append(s[486000:,3][i+1800])
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t1,C1, linewidth=1)
Z=np.linspace(-80,20,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.savefig("Fig2B.pdf")

In [ ]:
#Current
#IUNaV. INaV. INaP, IUNaP, IKNa, IKvhh, ILeak, ICaV
VK=-100.0
VLeak=-60.95
VNa=55.0
VCa=120.0

#INaV
def am(v):
    if v == -33.:
        return 1.
    else:
        return 0.1 * (v+33.0) / (1.0-np.exp(-(v+33.0)/10.0))
def bm(v):
    return 4.0 * np.exp(-(v+53.7)/12.0)

def mna3(v):
    xxx1xxx=am(v)/(am(v)+bm(v))
    return xxx1xxx**3

def INaV(g, v,h):
    return g*mna3(v)*h*(v-VNa)


#ILeak

def ILeak(g,v):
    return g*(v-VLeak)


def ILeakK(g,v):
    return g*0.6095*(v-VK)
def ILeakNa(g,v):
    return g*0.3905*(v)

#IKvhh

def IKvhh(g, v,n):
    return g*(n**4)*(v-VK)
                     
def mca(v):
    return (1.0 / (1.0 + np.exp(-(v+20.0)/9.0)))**2

def ICaV(g, v):
    return g*mca(v)*(v-VCa)


#IKNa
def mna(na):
    return 1.0/(1.0+(32.0/na)**(3.0))


def IKNa(g,v,na):
    return g*mna(na)*(v-VK)


#IUNaV

def amuv(v,x):
    if v == -33.:
        return 1.
    else:
        return 0.1 * (v+33.0+x) / (1.0-np.exp(-(v+33.0+x)/10.0))
def bmuv(v,x):
    return 4.0 * np.exp(-(v+53.7+x)/12.0)

def mnauv3(v,x):
    xxx1xxx=amuv(v,x)/(amuv(v,x)+bmuv(v,x))
    return xxx1xxx**3

def IUNaV(g,v,h,x):
    return g*mnauv3(v,x)*h*(v-VNa)



gK= 48.19198701
gUNaV=6.104226316
gKNa=9.65743873
gLeak=0.062345227
gCaV=0.391216425
tNa=6638.79306935
x=28.21858435
y=-7.96971366


In [ ]:
UNAV=[]
LENA=[]
CAV=[]

LEK=[]
KVHH=[]
KNA=[]

t=[]
for i in range(1000):
    t.append(0.001*0.1*i)
unavN=[]
lenaN=[]
cavN=[]
lekN=[]
kvhhN=[]
knaN=[]
Na=[]
tNAC=[]
IN=[]
for i in range(len(t)):
    UNAV.append(IUNaV(gUNaV, s[487800:,0][i], s[487800:,1][i], x))
    LENA.append(ILeakNa(gLeak, s[487800:,0][i]))
    CAV.append(ICaV(gCaV, s[487800:,0][i]))
    
    LEK.append(ILeakK(gLeak, s[487800:,0][i]))
    KVHH.append(IKvhh(gK, s[487800:,0][i], s[487800:,2][i]))
    KNA.append(IKNa(gKNa, s[487800:,0][i], s[487800:,3][i]))
    IN.append((-UNAV[i]-gLeak*(100-60.95)/100*0.44*(s[487800:,0][i]-55.0))/5000)

    tNAC.append(1000*s[487800:,3][i]/tNa)
    Na.append(s[487800:,3][i])
    
    unavN.append(abs(UNAV[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    lenaN.append(abs(LENA[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    cavN.append(abs(CAV[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    lekN.append(abs(LEK[i])/(abs(LEK[i])+abs(KVHH[i])+abs(KNA[i])))
    kvhhN.append(abs(KVHH[i])/(abs(LEK[i])+abs(KVHH[i])+abs(KNA[i])))
    knaN.append(abs(KNA[i])/(abs(LEK[i])+abs(KVHH[i])+abs(KNA[i])))
    
    
    


In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(t,IN, linewidth=1, label="IN", color="red")
ax2.plot(t,tNAC, linewidth=1, label="OUT", color="blue")
Z=np.linspace(0,0.1,5)
ax1.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax1.set_xticks(Z)
Z=np.linspace(1.130,1.16,4)
ax2.set_yticks(Z)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
plt.savefig("Fig2F.pdf")


In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,Na, linewidth=1, color="red")
Z=np.linspace(7.5,7.7,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.savefig("Fig2E.pdf")

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,unavN, linewidth=1, color="red", label="UNaV")
plt.plot(t,cavN, linewidth=1, color="orange", label="CaV")
plt.plot(t,lenaN, linewidth=1, color="yellow", label="LeakNa")
Z=np.linspace(0,1,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.legend()
plt.savefig("Fig2B.pdf")

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(t,lekN, linewidth=1, color="purple", label="LeakK")
plt.plot(t,kvhhN, linewidth=1, color="blue", label="Kvhh")
plt.plot(t,knaN, linewidth=1, color="green", label="KNa")
Z=np.linspace(0,1,5)
ax.set_yticks(Z)
Z=np.linspace(0,0.1,5)
ax.set_xticks(Z)
plt.legend()
plt.savefig("Fig2D.pdf")

In [ ]:
Tmini=[]
for i in range(100):
    Tmini.append(0.001*0.1*i)

In [ ]:
UNAV=[]
LENA=[]
CAV=[]

LEK=[]
KVHH=[]
KNA=[]


unavN=[]
lenaN=[]
cavN=[]
lekN=[]
kvhhN=[]
knaN=[]

tNAC=[]
IN=[]
for i in range(len(Tmini)):
    UNAV.append(IUNaV(gUNaV, s[487800:,0][i], s[487800:,1][i], x))
    LENA.append(ILeakNa(gLeak, s[487800:,0][i]))
    CAV.append(ICaV(gCaV, s[487800:,0][i]))
    
    LEK.append(ILeakK(gLeak, s[487800:,0][i]))
    KVHH.append(IKvhh(gK, s[487800:,0][i], s[487800:,2][i]))
    KNA.append(IKNa(gKNa, s[487800:,0][i], s[487800:,3][i]))
    IN.append((-UNAV[i]-LENA[i]*0.44)/1000)
    
    tNAC.append(s[487800:,3][i]/tNa)
    
    unavN.append(abs(UNAV[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    lenaN.append(abs(LENA[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    cavN.append(abs(CAV[i])/(abs(UNAV[i])+abs(LENA[i])+abs(CAV[i])))
    lekN.append(abs(LEK[i])/(abs(LEK[i])+abs(KVHH[i])+abs(KNA[i])))
    kvhhN.append(abs(KVHH[i])/(abs(LEK[i])+abs(KVHH[i])+abs(KNA[i])))
    knaN.append(abs(KNA[i])/(abs(LEK[i])+abs(KVHH[i])+abs(KNA[i])))
    
    
    

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(Tmini,unavN, linewidth=1, color="red", label="UNaV")
plt.plot(Tmini,cavN, linewidth=1, color="orange", label="CaV")
plt.plot(Tmini,lenaN, linewidth=1, color="yellow", label="LeakNa")
Z=np.linspace(0,1,2)
ax.set_yticks(Z)
Z=np.linspace(0,0.01,4)
ax.set_xticks(Z)
#plt.legend()
plt.savefig("Fig2Cmin.pdf")

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)
plt.plot(Tmini,lekN, linewidth=1, color="purple", label="LeakK")
plt.plot(Tmini,kvhhN, linewidth=1, color="blue", label="Kvhh")
plt.plot(Tmini,knaN, linewidth=1, color="green", label="KNa")
Z=np.linspace(0,1,2)
ax.set_yticks(Z)
Z=np.linspace(0,0.01,4)
ax.set_xticks(Z)
#plt.legend()
plt.savefig("Fig2Dmin.pdf")

In [ ]:
s0=s
U1=s0[483700:,0]
U2=s0[483700:,2]
U3=s0[483700:,3]
fig = plt.figure(figsize = (15, 15))
ax = fig.add_subplot(111, projection='3d')
ax.plot(U1, U2, U3, color="lightskyblue")
plt.savefig("Trajectory1.pdf")
plt.show()

In [ ]:
print(U1[100])
print(U2[100])
print(U3[100])
472500+14000

In [ ]:
U1=s0[472500:,0]
U2=s0[472500:,2]
U3=s0[472500:,3]

R1=[]
R2=[]
R3=[]

for i in range(14000):
    
    R1.append(U1[0+i])
    R2.append(U2[0+i])
    R3.append(U3[0+i])
    
R11=[]
R21=[]
R31=[]

for i in range(2500):
    
    R11.append(U1[11000+i])
    R21.append(U2[11000+i])
    R31.append(U3[11000+i])
R12=[]
R22=[]
R32=[]

for i in range(2200):
    
    R12.append(U1[14000+i])
    R22.append(U2[14000+i])
    R32.append(U3[14000+i])
fig = plt.figure(figsize = (15, 15))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel("V (mV)", size = 14)
ax.set_ylabel("nK (unitless)", size = 14)
ax.set_zlabel("Na (mM)", size = 14)

Z=np.linspace(-80,60,5)
ax.set_xticks(Z)
Z=np.linspace(0,1,5)
ax.set_yticks(Z)
Z=np.linspace(6.8,7.8,5)
ax.set_zticks(Z)
ax.plot(R1, R2, R3, color="red")

ax.plot(R12, R22, R32, color="lightskyblue", linewidth=1)

plt.savefig("Trajectory1.pdf")
plt.show()
    